In [1]:
import pandas as pd
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt
import seaborn as sns
import math
import time

1) https://translated.turbopages.org/proxy_u/en-ru.ru.a44a2a24-65c27e99-65cf9c32-74722d776562/https/www.baeldung.com/cs/gradient-descent-logistic-regression

2) https://www.evogeek.ru/articles/40066/

<center><h1> EDA </h1><center>

In [2]:
iris = datasets.load_iris()

df = pd.DataFrame(iris.data, columns=iris.feature_names)
df['species'] = [iris.target_names[i] for i in iris.target] 
df = df.loc[df['species'] != 'setosa'].copy()
# не стал подключать LabelEncoder
df.loc[df['species'] == 'versicolor', 'species'] = 0 
df.loc[df['species'] == 'virginica', 'species'] = 1

df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),species
50,7.0,3.2,4.7,1.4,0
51,6.4,3.2,4.5,1.5,0
52,6.9,3.1,4.9,1.5,0
53,5.5,2.3,4.0,1.3,0
54,6.5,2.8,4.6,1.5,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,1
146,6.3,2.5,5.0,1.9,1
147,6.5,3.0,5.2,2.0,1
148,6.2,3.4,5.4,2.3,1


<center><h1> Choosing variables </h1><center>

In [3]:
X = np.array(df[['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']])
y = df['species']

<center><h1> Main functions </h1><center>

In [4]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x.astype(float)))
    
def logloss(y, y_proba):
    logloss_1 = np.sum(np.log(y_proba[y == 1] + 1e-50))
    logloss_0 = np.sum(np.log(1 - y_proba[y == 0] + 1e-50))
    logloss_total = -(logloss_0 + logloss_1) / len(y)
    return logloss_total
    
def grad_logloss(X, y, theta):
    y_proba = sigmoid(X @ theta)
    return X.T @ (y_proba - y), y_proba


<center><h1> Optimize functions </h1><center>

In [5]:
class Optimize():
    def __init__(self):
        self = self
    
    def finish_checker(self, X, y, theta, i, Print_iter=False):
        y_proba = grad_logloss(X, y, theta)[1]
        yhat = np.where(y_proba >= 0.5, 1, 0)
        accuracy =  (yhat == y).sum() / len(y)
        
        if Print_iter:
            if i % 50 == 0:
                print(f'Iteration: {i}')
                print(f'LogLoss: {logloss(y, y_proba)}')
                print(f'Accuracy: {accuracy} \n')
        
        if accuracy >= 0.95:
#             print(f'LogLoss on the last iteration: {logloss(y, y_proba)}')
#             print(f'Iterations till 95% accuracy: {i}')
            return True
        
    def gradient_descent(self, X, y, n=1000, learning_rate=0.001, eps=0.0001, Print_iter=True):
        start_time = time.time()
        
        X = np.c_[np.ones(len(X)), X]
        theta = np.random.randn(X[1].shape[0])
        
        for i in range(n):
            theta = theta - learning_rate * grad_logloss(X, y, theta)[0]
            
            if self.finish_checker(X, y, theta, i, Print_iter):
                break
        return i, time.time() - start_time
        
    def nesterov(self, X, y, n=1000, learning_rate=0.001, eps=0.0001 ,yps=0.9, Print_iter=False):
        start_time = time.time()
        
        X = np.c_[np.ones(len(X)), X]
        theta = np.random.randn(X[1].shape[0])
        Vt = np.zeros(X[1].shape)
        
        for i in range(n):
            impulse = theta - (yps * Vt)
            Vt = (yps * Vt) + (learning_rate * grad_logloss(X, y, impulse)[0])
            theta = theta - Vt
            
            if self.finish_checker(X, y, theta, i, Print_iter):
                break
        return i, time.time() - start_time

    def RMSProp(self, X, y, n=1000, learning_rate=0.001, eps=0.0000001 ,yps=0.9, Print_iter=False):
        start_time = time.time()
        
        X = np.c_[np.ones(len(X)), X]
        theta = np.random.randn(X[1].shape[0])
        Egt = np.zeros(X[1].shape)
        
        for i in range(n):
            g = grad_logloss(X, y, theta)[0]
            
            Egt = (yps * Egt) + (1 - yps) * (g**2)
            theta = theta - (g * (learning_rate / np.sqrt(Egt.astype(float) + eps)))
            
            if self.finish_checker(X, y, theta, i, Print_iter):
                break
        return i, time.time() - start_time

<center><h1> Calling different models </h1><center>

In [6]:
model = Optimize()

In [7]:
# start_time = time.time()
gradient_descent = model.gradient_descent(X, y, n=5000, Print_iter=False)
# print("\n--- %s seconds ---" % (time.time() - start_time))

In [8]:
nesterov = model.nesterov(X, y, n=5000, Print_iter=False)

In [30]:
RMSProp = model.RMSProp(X, y, n=5000, yps=0.00015, Print_iter=False)

<center><h1> DataFrame of results </h1><center>

In [31]:
result = pd.DataFrame(index = ('GD', 'RMSProp', 'Nesterov'), columns=('Итераций до 95% точности (метрика)', 'Time'))

l = [gradient_descent, RMSProp, nesterov]

for m in range(len(l)):
    result.iloc[m] = l[m]
    
result

,Итераций до 95% точности (метрика),Time
GD,61,0.053053
RMSProp,1146,0.781521
Nesterov,24,0.023145


<center><h1> Выводы </h1><center>

1) Какая-то ерунда с RMSProp, хотя другие модели, кажется, работают, как нужно.
2) Нестеров оказался самым быстрым и точным (подразумеваю, что это не только потому, что RMSProp не работает)
3) Сейчас intercept занесён в матрицу иксов; в идеале, насколько я понимаю, его также рассчитывать отдельно, как и тета (aka w0)

4) В ML коммюнити нет договорённостей об обозначениях переменных в уравнениях!!!!!!!
5) Механизм прерывания цикла вышел очень кривой, нужно над этим поработать

<h2> Подскажите, пожалуйста, в чём ошибка с RMSProp? </h2> Нужно было работать не с матрицами, а с раномными чанками или значениями х и у?